# Gradiente ascendente sin vectores

Este notebook implementa el gradiente ascendente usando ciclos y operaciones elemento a elemento. El objetivo es entender cómo se construye el gradiente y cómo se actualizan los parámetros del modelo de forma explícita.

## Imports y datos

In [9]:
import numpy as np
import math

# Ejemplo de conjunto de datos pequeño
# Cada fila de X representa un ejemplo y cada columna una característica (feature)
X = np.array([
    [0, 0, 1, 0],
    [1, 1, 1, 1],
    [0, 0, 0, 1]
])

# Etiquetas reales (Y[i] = 1 para clase positiva, 0 para clase negativa)
Y = np.array([1, 0, 1])


## Función sigmoide

Definimos la función sigmoide como función de activación. 
Toma un valor escalar y regresa un valor entre 0 y 1, interpretado como probabilidad.

In [10]:
def sigmoid(z):
    """
    Calcula la función sigmoide para un valor escalar z.

    Parámetros:
        z: número real (float)

    Regresa:
        float en el intervalo (0, 1)
    """
    return 1.0 / (1.0 + np.exp(-z))


## Cálculo de probabilidades

Calcula, para cada ejemplo, la probabilidad de que Y = 1 dada la combinación lineal de características.  
Se usa la función sigmoide aplicada a la suma ponderada de entradas y parámetros (β).

In [11]:
def calcular_probabilidades(X, beta):
    """
    Calcula las probabilidades predichas para cada ejemplo.

    Fórmula:
        p[i] = P(Y[i]=1 | X[i]) = sigmoid(sum_j beta[j] * X[i][j])

    Parámetros:
        X     : matriz de características (m x n)
        beta  : lista o vector con los parámetros actuales del modelo (longitud n)

    Regresa:
        Lista con las probabilidades p[i] para cada ejemplo
    """
    m = len(X)
    n = len(beta)

    # Inicializar lista de probabilidades con ceros
    p = [0.0 for _ in range(m)]

    # Calcular z = β·x para cada ejemplo y aplicar la sigmoide
    for i in range(m):
        z = 0.0
        for j in range(n):
            z += beta[j] * X[i][j]
        p[i] = sigmoid(z)

    return p

## Cálculo del gradiente

El gradiente se obtiene sumando la contribución de cada ejemplo.  
Para cada parámetro β_j, se acumula la cantidad `(Y[i] - p[i]) * X[i][j]`.

In [12]:
def calcular_gradiente(X, Y, p):
    """
    Calcula el gradiente de la función de verosimilitud.

    Fórmula:
        grad[j] = sum_i (Y[i] - p[i]) * X[i][j]

    Parámetros:
        X : matriz de características (m x n)
        Y : vector de etiquetas reales
        p : lista de probabilidades predichas

    Regresa:
        grad : lista con la derivada parcial de cada parámetro β_j
    """
    n = len(X[0])
    m = len(X)

    grad = [0.0 for _ in range(n)]

    # Para cada parámetro beta_j
    for j in range(n):
        suma = 0.0
        # Sumar las contribuciones de cada ejemplo
        for i in range(m):
            suma += (Y[i] - p[i]) * X[i][j]
        grad[j] = suma

    return grad


## Norma Euclidiana del gradiente

Se usa para medir el tamaño del vector gradiente.  
Permite definir una condición de paro cuando el gradiente es suficientemente pequeño.

In [13]:
def norma_vector(v):
    """
    Calcula la norma Euclidiana de un vector.

    Fórmula:
        ||v||_2 = sqrt(sum_j v[j]^2)

    Parámetros:
        v : lista o vector numérico

    Regresa:
        Valor escalar con la norma Euclidiana de v
    """
    suma = 0.0
    for i in range(len(v)):
        suma += v[i] * v[i]
    return math.sqrt(suma)


## Actualización de parámetros β

Se actualiza cada parámetro en dirección del gradiente.

In [14]:
def actualizar_beta(beta, grad, eta):
    """
    Actualiza los parámetros beta usando la regla del gradiente ascendente.

    Fórmula:
        beta_j^{nuevo} = beta_j^{viejo} + eta * grad[j]

    Parámetros:
        beta : lista de parámetros actuales
        grad : gradiente calculado
        eta  : tasa de aprendizaje

    Regresa:
        Lista beta actualizada
    """
    n = len(beta)
    for j in range(n):
        beta[j] += eta * grad[j]
    return beta


## Algoritmo completo de gradiente ascendente

Integra todas las funciones anteriores:
1. Calcula probabilidades.
2. Calcula gradiente.
3. Evalúa la norma del gradiente.
4. Actualiza parámetros hasta converger o alcanzar el máximo de iteraciones.

In [15]:
def gradiente_ascendente(X, Y, eta=0.01, max_iter=1000, tol=0.0001):
    """
    Implementa el algoritmo completo de gradiente ascendente.

    Parámetros:
        X         : matriz de características (m x n)
        Y         : vector de etiquetas (longitud m)
        eta       : tasa de aprendizaje
        max_iter  : número máximo de iteraciones
        tol       : tolerancia para detener cuando ||grad|| < tol

    Regresa:
        beta : lista con los parámetros ajustados
    """
    n = len(X[0])
    beta = [0.0 for _ in range(n)]  # inicialización de parámetros

    for iteration in range(max_iter):
        # 1. Calcular probabilidades
        p = calcular_probabilidades(X, beta)

        # 2. Calcular gradiente
        grad = calcular_gradiente(X, Y, p)

        # 3. Calcular norma del gradiente
        grad_norm = norma_vector(grad)

        if grad_norm < tol:
            print(f"Convergió en {iteration} iteraciones")
            break

        # 4. Actualizar parámetros
        beta = actualizar_beta(beta, grad, eta)

    return beta

## Entrenamiento del modelo y resultado final

In [16]:
betas_finales = gradiente_ascendente(X, Y, eta=0.01, max_iter=5000, tol=1e-5)

print("Parámetros finales aprendidos (beta):")
print(betas_finales)

Parámetros finales aprendidos (beta):
[np.float64(-4.607455855430737), np.float64(-4.607455855430737), np.float64(2.892886219289177), np.float64(2.892886219289177)]
